In [2]:
import xarray as xr

# select location of interest
lat = 61.8474
lon = 24.2948

ds_era5land = xr.open_mfdataset("C:/STEMMUS_SCOPE_data/global/era5-land/*.nc").sel(latitude=lat, longitude=lon, method='nearest').compute()
ds_era5 = xr.open_mfdataset("C:/STEMMUS_SCOPE_data/global/era5/*.nc").sel(latitude=lat, longitude=lon, method='nearest').compute()

ds_era5land = ds_era5land.resample(time="1800S").interpolate('linear')
ds_era5land = ds_era5land.drop(['latitude', 'longitude'])

ds_era5 = ds_era5.resample(time="1800S").interpolate('linear')
ds_era5 = ds_era5.drop(['latitude', 'longitude'])

ds = xr.merge([ds_era5land, ds_era5])


In [3]:
ds

<xarray.Dataset>
Dimensions:  (time: 192863)
Coordinates:
  * time     (time) datetime64[ns] 2010-01-01 ... 2020-12-31T23:00:00
Data variables:
    d2m      (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    t2m      (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    mtpr     (time) float64 1.537e-05 1.614e-05 1.691e-05 ... 0.0001955 0.00019
    sp       (time) float64 9.88e+04 9.882e+04 9.885e+04 ... 9.909e+04 9.909e+04
    ssrd     (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.125 0.125 0.125 0.125
    strd     (time) float64 8.856e+05 8.826e+05 ... 1.083e+06 1.081e+06
    u10      (time) float64 -3.31 -3.323 -3.336 -3.257 ... 1.076 1.433 1.79
    v10      (time) float64 -1.994 -1.892 -1.789 ... 0.08913 -4.712e-05 -0.08922
Attributes:
    Conventions:  CF-1.6
    history:      2023-01-16 10:39:50 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

Calculate required variables

In [4]:
from PyStemmusScope import variable_conversion as vc

ds_ss = xr.Dataset()
ds_ss["wind_speed"] = (ds["u10"] ** 2 + ds["v10"] ** 2) ** 0.5
ds_ss["t_air_celcius"] = ds["t2m"] - 273.15  # K -> degC
ds_ss["precip"] = ds["mtpr"] / 10  # mm/s -> cm/s
ds_ss["p_surf"] = ds["sp"] / 1000  # Pa -> kPa
ds_ss["sw_down"] = ds["ssrd"] / 3600  # J * hr / m2 ->  W / m2
ds_ss["lw_down"] = ds["strd"] / 3600  # J * hr / m2 ->  W / m2

ds_ss["ea"] = vc.calculate_es(ds["d2m"] - 273.15)
ds_ss["vpd"] = vc.calculate_es(ds_ss["t_air_celcius"]) - ds_ss["ea"]
ds_ss["rh"] = ds_ss["ea"] / vc.calculate_es(ds_ss["t_air_celcius"])  
ds_ss["qair"] = vc.specific_humidity(ds_ss["ea"], ds_ss["p_surf"])

In [5]:
ds_ss

<xarray.Dataset>
Dimensions:        (time: 192863)
Coordinates:
  * time           (time) datetime64[ns] 2010-01-01 ... 2020-12-31T23:00:00
Data variables:
    wind_speed     (time) float64 3.864 3.824 3.785 3.713 ... 1.079 1.433 1.792
    t_air_celcius  (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    precip         (time) float64 1.537e-06 1.614e-06 ... 1.955e-05 1.9e-05
    p_surf         (time) float64 98.8 98.82 98.85 98.87 ... 99.08 99.09 99.09
    sw_down        (time) float64 0.0 0.0 0.0 ... 3.472e-05 3.472e-05 3.472e-05
    lw_down        (time) float64 246.0 245.2 244.3 245.3 ... 301.1 300.7 300.4
    ea             (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    vpd            (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    rh             (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    qair           (time) float64 nan nan nan nan nan ... nan nan nan nan nan